# Data Setup

In [ ]:
import torch

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [80]:
if torch.cuda.is_available():
    print("GPU available.")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
elif torch.mps.is_available():
    print("MPS available.")
    print(f"Using MPS: {torch.mps.device_count()}")
    device = torch.device("mps")
else:
    print("Using CPU.")
    device = torch.device("cpu")

device = torch.device("cpu")

torch.set_default_device(device=device)

MPS available.
Using MPS: 1


In [81]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [82]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)

In [83]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [84]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [85]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [86]:
x_train_tensor = torch.from_numpy(x_train).to(dtype=torch.float32, device=device)
x_test_tensor = torch.from_numpy(x_test).to(dtype=torch.float32, device=device)
y_train_tensor = torch.from_numpy(y_train).to(dtype=torch.float32, device=device)
y_test_tensor = torch.from_numpy(y_test).to(dtype=torch.float32, device=device)

# Defining Model

In [87]:
class NeuralNetwork():

    def __init__(self, x: torch.Tensor):
        self.weights = torch.rand(x.shape[1], 1, dtype = torch.float32, requires_grad = True, device=device)
        self.bias = torch.zeros(1, 1, dtype = torch.float32, requires_grad = True, device=device)
    
    def forward(self, x: torch.Tensor):
        z = torch.matmul(x, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred
    
    def loss_function(self, y_pred: torch.Tensor, y: torch.Tensor):
        #Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        # Calculate loss
        loss = -(y* torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
        return loss

# Parameters

In [88]:
lr = 0.1
epochs = 1000

# Training Pipeline

In [89]:
model = NeuralNetwork(x_train_tensor)

# Define loop
for epoch in range(epochs):
    # Forward Pass
    y_pred = model.forward(x_train_tensor)

    # Loss Calculation
    loss = model.loss_function(y_pred, y_train_tensor)
    print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")

    # Backward Pass
    loss.backward()

    # Paramers Update
    with torch.no_grad():
        model.weights -= lr * model.weights.grad
        model.bias -= lr * model.bias.grad
    
    # Zero Gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()

Epoch: 1, Loss: 3.207179307937622
Epoch: 2, Loss: 3.0473341941833496
Epoch: 3, Loss: 2.8788464069366455
Epoch: 4, Loss: 2.7082901000976562
Epoch: 5, Loss: 2.5326600074768066
Epoch: 6, Loss: 2.351226329803467
Epoch: 7, Loss: 2.1679840087890625
Epoch: 8, Loss: 1.9861539602279663
Epoch: 9, Loss: 1.8063030242919922
Epoch: 10, Loss: 1.6349130868911743
Epoch: 11, Loss: 1.468701958656311
Epoch: 12, Loss: 1.3158557415008545
Epoch: 13, Loss: 1.1793692111968994
Epoch: 14, Loss: 1.0622143745422363
Epoch: 15, Loss: 0.9666118025779724
Epoch: 16, Loss: 0.8932411670684814
Epoch: 17, Loss: 0.8404352068901062
Epoch: 18, Loss: 0.8043656349182129
Epoch: 19, Loss: 0.7803551554679871
Epoch: 20, Loss: 0.7642470598220825
Epoch: 21, Loss: 0.7530145645141602
Epoch: 22, Loss: 0.744718611240387
Epoch: 23, Loss: 0.7382098436355591
Epoch: 24, Loss: 0.7328335046768188
Epoch: 25, Loss: 0.72822105884552
Epoch: 26, Loss: 0.7241616249084473
Epoch: 27, Loss: 0.720528781414032
Epoch: 28, Loss: 0.7172425389289856
Epoch: 2

# Evaluation

In [90]:
with torch.no_grad():
    y_pred = model.forward(x_test_tensor)
    y_pred = (y_pred > 0.7).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.6315789222717285
